In [ ]:
# Switch to Tensorflow 1.x
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt
import numpy as np

# Data processing 

In [ ]:
(X_train, Y_train), (X_test, Y_test) = K.datasets.cifar10.load_data()

In [ ]:
Y_train.shape

(50000, 1)

In [ ]:
Y_train[0]

array([6], dtype=uint8)

In [ ]:
def preprocess_data(X, Y):
  x = tf.keras.applications.inception_resnet_v2.preprocess_input(X)
  y = K.utils.to_categorical(Y, 10)
  return x, y

In [ ]:
X_train, Y_train = preprocess_data(X_train, Y_train)
X_test, Y_test = preprocess_data(X_test, Y_test)

# Transfer Learning

In [ ]:
base_model = K.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [ ]:
base_model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 149, 149, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, 149, 149, 32) 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 149, 149, 32) 0           batch_normalization_203[0][0]    
________________________________________________________________________________

In [ ]:
inputs = K.Input(shape=(32,32,3))
input = K.layers.Lambda(lambda image : tf.image.resize(image, (299, 299)))(inputs)

x = base_model(input, training=False)

x = K.layers.GlobalAveragePooling2D()(x)

x = K.layers.Dense(500, activation='relu')(x)
x = K.layers.Dropout(0.3)(x)
outputs = K.layers.Dense(10, activation='softmax')(x)

model = K.Model(inputs, outputs)



In [ ]:
base_model.trainable = False
optimizer = K.optimizers.Adam()

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["acc"])


In [ ]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test),batch_size=300, epochs=4, verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/4
50000/50000 [==============================] - 577s 12ms/sample - loss: 0.4075 - acc: 0.8684 - val_loss: 0.2938 - val_acc: 0.9036
Epoch 2/4
50000/50000 [==============================] - 540s 11ms/sample - loss: 0.2946 - acc: 0.9020 - val_loss: 0.2717 - val_acc: 0.9115
Epoch 3/4
50000/50000 [==============================] - 540s 11ms/sample - loss: 0.2663 - acc: 0.9094 - val_loss: 0.2649 - val_acc: 0.9137
Epoch 4/4
50000/50000 [==============================] - 540s 11ms/sample - loss: 0.2485 - acc: 0.9153 - val_loss: 0.2632 - val_acc: 0.9129


In [ ]:
results = model.evaluate(X_test, Y_test)
print("test loss, test acc: ",results)

10000/10000 [==============================] - 97s 10ms/sample - loss: 0.2632 - acc: 0.9129
test loss, test acc:  [0.2631672409176826, 0.9129]


# Fine Tuning
Steps for fine-tuning a network are as follow:

1. Add your custom network on top of an already trained base network.


2. Freeze the base network.


3. Train the part you added.


4. Unfreeze some layers in the base network.


5. Jointly train both these layers and the part you added.

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(base_model, to_file='model.png')

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.591003 to fit



In [ ]:
for i, layer in enumerate(base_model.layers):
  print(i, layer.name)

0 input_6
1 conv2d_203
2 batch_normalization_203
3 activation_203
4 conv2d_204
5 batch_normalization_204
6 activation_204
7 conv2d_205
8 batch_normalization_205
9 activation_205
10 max_pooling2d_4
11 conv2d_206
12 batch_normalization_206
13 activation_206
14 conv2d_207
15 batch_normalization_207
16 activation_207
17 max_pooling2d_5
18 conv2d_211
19 batch_normalization_211
20 activation_211
21 conv2d_209
22 conv2d_212
23 batch_normalization_209
24 batch_normalization_212
25 activation_209
26 activation_212
27 average_pooling2d_1
28 conv2d_208
29 conv2d_210
30 conv2d_213
31 conv2d_214
32 batch_normalization_208
33 batch_normalization_210
34 batch_normalization_213
35 batch_normalization_214
36 activation_208
37 activation_210
38 activation_213
39 activation_214
40 mixed_5b
41 conv2d_218
42 batch_normalization_218
43 activation_218
44 conv2d_216
45 conv2d_219
46 batch_normalization_216
47 batch_normalization_219
48 activation_216
49 activation_219
50 conv2d_215
51 conv2d_217
52 conv2d_220

In [ ]:
for layer in base_model.layers[:402]:
  layer.trainable=False

for layer in base_model.layers[402:]:
  layer.trainable=True

In [ ]:
optimizer=K.optimizers.Adam(1e-5)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['acc'])

In [ ]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=300, epochs=4, verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/4
50000/50000 [==============================] - 544s 11ms/sample - loss: 0.2212 - acc: 0.9251 - val_loss: 0.2464 - val_acc: 0.9177
Epoch 2/4
50000/50000 [==============================] - 540s 11ms/sample - loss: 0.2099 - acc: 0.9277 - val_loss: 0.2443 - val_acc: 0.9189
Epoch 3/4
50000/50000 [==============================] - 540s 11ms/sample - loss: 0.2071 - acc: 0.9297 - val_loss: 0.2430 - val_acc: 0.9192
Epoch 4/4
50000/50000 [==============================] - 540s 11ms/sample - loss: 0.2041 - acc: 0.9303 - val_loss: 0.2418 - val_acc: 0.9202
